In [38]:
import numpy as np
import pandas as pd
import json 

import bokeh
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.sampledata.commits import data
from bokeh.transform import jitter

bokeh.io.output_notebook()

Loading BokehJS ...

In [39]:
df = pd.read_csv('../Lung_Paper_Images/Quantification/Lung/20200630_Stitched_Lung_Cell_Counts.csv', comment='#')
df.head()

,Date,Size Threshold,Gaussian Size,Truncation,Intensity Threshold,Minimum Size,Virus,Animal,Replicate,Count,Cells Quantified,Brightness List,Applied Threshold,Image Multiplication Factor,Minimum Pixel Value,Maximum Pixel Value,Area Threshold,Total Area
0,2020-06-30,0.62,5,2,1000,10,AAV5,1,1,84,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.09374644248769036, '2': 0.396884378149...",0.301449,0.706774,-0.184533,0.527600,0.020282,3.127379e+07
1,2020-06-30,0.62,5,2,1000,10,AAV5,1,2,105,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.34954984628418456, '2': 0.301035344397...",0.322450,0.699570,-0.204931,0.543257,0.019231,3.291692e+07
2,2020-06-30,0.62,5,2,1000,10,AAV5,2,1,101,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.040592697357633566, '2': 0.07469444680...",0.243228,0.591669,-0.151784,0.571073,0.020107,4.802036e+07
3,2020-06-30,0.62,5,2,1000,10,AAV5,2,2,108,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.08277810712698083, '2': 0.062237664806...",0.396656,0.584836,-0.267212,0.474624,0.019407,4.244302e+07
4,2020-06-30,0.62,5,2,1000,10,AAV5,3,1,28,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.07123424141933186, '2': 0.198157759863...",0.411377,0.589563,-0.211362,0.348148,0.019425,2.645178e+07


In [40]:
# Using Determine the transduction as a function of tissue area
df['Cells Transduced per mm2'] = df['Count'] / df['Total Area'] * 1000000

# Get the median brightness from the brightness dictionary
for inds in np.arange(0, len(df)):     
    # Load the brightness dictionary, replacing the single quotation marks in the keys with double quotation marks for json compliance. 
    brightness_dict = json.loads(df.loc[inds, 'Brightness List'].replace("\'", "\""))
    
    # Determine the median of each brightness dictionary and store it in the dataframe
    df.loc[inds, 'Brightness Median'] = np.median(list(brightness_dict.values()))

# Rename the specific rows for plotting purposes. 
df['Virus'] = df['Virus'].replace('CAPA4','AAV.CAP-A4')

In [41]:
#Group the data by the specific virus and animals
grouped = df.groupby(['Virus','Animal'])

# Make two new dataframes that track cell transduction and median cell brightness respectively
df_counts = grouped['Cells Transduced per mm2'].mean().reset_index()
df_brightness = grouped['Brightness Median'].mean().reset_index()

In [42]:
#Initialize an output file to save an html of the generated plot
#output_file("../Lung_Paper_Images/Quantification/Lung/lung_transduction_per_sq_mm.html")

# Store the data for plotting
source = ColumnDataSource(df_counts)

# Create the catagories to plot from
catagories = list(df['Virus'].unique())

# Intitialize a figure
p = figure(plot_width=300, plot_height=300, x_range=catagories, y_axis_label = 'Cells Transduced per mm\u00b2')

# Plot the data as a scatter plot
p.circle(x=jitter('Virus', width=0.6, range=p.x_range), y='Cells Transduced per mm2', source=source, alpha=1)

# Simply modify plot characteristics 
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'
show(p)

In [43]:
#Initialize an output file to save an html of the generated plot
output_file("../Lung_Paper_Images/Quantification/Lung/lung_median_brightness.html")

# Store the data for plotting
source = ColumnDataSource(df_brightness)

# Create the catagories to plot from
catagories = list(df['Virus'].unique())

# Intitialize a figure
p = figure(plot_width=300, plot_height=300, x_range=catagories, y_axis_label = 'Median Cell Brightness', title="Lung")

# Plot the data as a scatter plot
p.circle(x=jitter('Virus', width=0.6, range=p.x_range), y='Brightness Median', source=source, alpha=1)

p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

show(p)